In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from six.moves import xrange

In [2]:
INPUT_SIZE = 784
LAYOUT_1_SIZE = 500
OUTPUT_SIZE = 10

In [3]:
def get_weight_variable(shape):
    weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    return weights

In [4]:
def inference(input_tensor):
    with tf.variable_scope("layer_1", reuse=tf.AUTO_REUSE):
        weights = get_weight_variable([INPUT_SIZE, LAYOUT_1_SIZE])
        biases = tf.get_variable("biases", [LAYOUT_1_SIZE], initializer=tf.constant_initializer(0.0))
        layer_1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
    with tf.variable_scope("layer_2", reuse=tf.AUTO_REUSE):
        weights = get_weight_variable([LAYOUT_1_SIZE, OUTPUT_SIZE])
        biases = tf.get_variable("biases", [OUTPUT_SIZE], initializer=tf.constant_initializer(0.0))
        logits = tf.matmul(layer_1, weights) + biases
    return logits

In [5]:
def compute_loss(logits, labels):
    corss_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
    return tf.reduce_mean(corss_entropy)

In [6]:
def train(loss, learning_rate):
    global_step = tf.Variable(0, trainable=False)
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    return train_op

In [7]:
def evaluation(logits, labels):
    correct = tf.nn.in_top_k(logits, labels, 1)
    return tf.reduce_sum(tf.cast(correct, tf.int32))

In [8]:
BATCH_SIZE = 100
MAX_STEPS = 2000
LEARNING_RATE = 0.1
INPUT_DATA_DIR = "./data"

In [9]:
x = tf.placeholder(tf.float32, shape=[None, INPUT_SIZE])
y_label = tf.placeholder(tf.float32, shape=[None, OUTPUT_SIZE])
logits = inference(x)
loss = compute_loss(logits, tf.argmax(y_label, 1))
train_op = train(loss, LEARNING_RATE)
correct = evaluation(logits, tf.argmax(y_label, 1))

init = tf.global_variables_initializer()

In [10]:
mnist = input_data.read_data_sets("./data", one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [11]:
with tf.Session() as sess:
    sess.run(init)
    for step in xrange(1, MAX_STEPS + 1):
        xs, ys = mnist.train.next_batch(BATCH_SIZE)
        _, loss_value = sess.run([train_op, loss], feed_dict={x: xs, y_label: ys})
        if step % 100 == 0:
            x_validation = mnist.validation.images
            y_validation = mnist.validation.labels
            correct_value = sess.run(correct, feed_dict={x: x_validation, y_label: y_validation})
            precision =  correct_value / mnist.validation.num_examples
            print("Step %d, loss = %g, validation set precision = %g" % (step, loss_value, precision))

Step 100, loss = 0.438273, validation set precision = 0.8918
Step 200, loss = 0.504826, validation set precision = 0.9126
Step 300, loss = 0.284184, validation set precision = 0.9198
Step 400, loss = 0.368827, validation set precision = 0.9288
Step 500, loss = 0.215905, validation set precision = 0.9348
Step 600, loss = 0.116425, validation set precision = 0.9384
Step 700, loss = 0.211547, validation set precision = 0.9434
Step 800, loss = 0.27161, validation set precision = 0.9456
Step 900, loss = 0.290639, validation set precision = 0.9506
Step 1000, loss = 0.200191, validation set precision = 0.95
Step 1100, loss = 0.160943, validation set precision = 0.9502
Step 1200, loss = 0.120641, validation set precision = 0.955
Step 1300, loss = 0.328334, validation set precision = 0.956
Step 1400, loss = 0.161116, validation set precision = 0.9574
Step 1500, loss = 0.169556, validation set precision = 0.9596
Step 1600, loss = 0.129965, validation set precision = 0.9598
Step 1700, loss = 0.15